https://github.com/openai/openai-cookbook/blob/main/examples/vector_databases/qdrant/Getting_started_with_Qdrant_and_OpenAI.ipynb

MIT License

Copyright (c) 2025 OpenAI

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# 使用 OpenAI 向量嵌入的向量資料庫

本筆記本將逐步引導你如何使用 **OpenAI 向量嵌入（embeddings）** 

## 流程如下

1. 下載已經整理好的資料集.csv
2. 選擇需要做相似度搜尋的資料集內容
3. 使用 **OpenAI API** 傳入資料集字串，回傳計算好的向量嵌入
4. 將計算完成的嵌入儲存到資料集.csv，提供下節 [2_Embedding_Search_with_Qdrant_and_OpenAI.ipynb](./2_Embedding_Search_with_Qdrant_and_OpenAI.ipynb) RAG 系統使用

---

## 什麼是嵌入（embeddings）？

https://platform.openai.com/docs/guides/embeddings

OpenAI 的文字嵌入技術用來衡量文字字串之間的相關性。嵌入常被應用於以下場景：

* **搜尋**（根據與查詢字串的相關性來排序結果）
* **分群**（將相似的文字字串歸為一類）
* **推薦**（推薦與某些文字內容相關的項目）
* **異常偵測**（找出與其他內容關聯度低的異常項目）
* **多樣性分析**（分析文字之間的相似性分佈）
* **分類**（將文字字串分類到最相似的標籤中）

嵌入是一個由浮點數組成的向量（列表）。兩個向量之間的距離可以用來衡量它們的相關性。距離越小，表示相關性越高；距離越大，表示相關性越低。

---

## 如何取得嵌入（embedding）

要取得嵌入向量，將你的文字字串與嵌入模型名稱（例如 `text-embedding-3-small`）一起傳送到 Embeddings API 的端點。

回應會包含嵌入向量（由浮點數組成的列表）以及其他一些中繼資料。你可以擷取嵌入向量，將其儲存到向量資料庫中，並用於各種應用情境。


```python
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)
```

以下是 API 回應範例：

```json
{
  "object": "list",
  "data": [
    {
      "object": "embedding",
      "index": 0,
      "embedding": [
        -0.006929283495992422,
        -0.005336422007530928,
        -4.547132266452536e-05,
        -0.024047505110502243
      ]
    }
  ],
  "model": "text-embedding-3-small",
  "usage": {
    "prompt_tokens": 5,
    "total_tokens": 5
  }
}
```

預設情況下，`text-embedding-3-small` 模型產生的向量長度為 1536，`text-embedding-3-large` 模型為 3072。如果你希望在不喪失語意表示能力的前提下壓縮向量維度，可以使用 `dimensions` 參數。關於嵌入維度的詳細資訊，可以參考「嵌入使用情境」章節。

---

## 使用基於嵌入的搜尋進行問答（Question Answering）

在許多常見情況中，模型並未訓練於包含關鍵事實與資訊的資料上，而這些資料對你回應使用者查詢是非常重要的。為了解決這個問題，一種常見的方法是：**將額外的資訊放入模型的上下文視窗（context window）中**。

這種方式在許多使用場景中是有效的，但缺點是會導致 **更高的 token 成本**。

在這份筆記中（notebook），我們使用 2 的做法

1. **直接將資訊放入 context 中（高 token 成本）**
2. **使用嵌入技術進行語意搜尋，只取出相關段落放入 context 中（節省 token 並保持相關性）**

這種基於嵌入的問答方式，通常包括以下步驟：

* 將知識庫中的文件文字轉換為嵌入向量
* 將使用者查詢轉換為嵌入向量
* 計算查詢與文件段落之間的相似度
* 根據相似度擷取前幾個最相關段落
* 將這些段落與使用者查詢一起放入模型 context 中，進行回答生成

這樣的方式在建構 **私有知識庫問答系統（例如 RAG：Retrieval-Augmented Generation）** 時特別有用。


### 📦 安裝必要套件

本筆記本需要安裝 `openai` 主要套件，此外還會用到一些其他輔助的函式庫。

#### 套件說明：

* **`openai`**：用於與 OpenAI API 溝通，產生文字嵌入（embeddings）。
* **`tqdm`**：提供進度條，方便追蹤資料處理過程。
* **`datasets`**：來自 Hugging Face，用於下載與載入示範資料集。

安裝完成後，就可以開始進行後續的操作了。

In [1]:
! pip install openai pandas wget kagglehub tqdm tenacity tiktoken ipywidgets

### 🔐 準備你的 OpenAI API 金鑰

OpenAI API 金鑰會用來將文件與查詢轉換為向量（vectorization）。

如果你還沒有 OpenAI API 金鑰，可以從這裡申請：
👉 [https://platform.openai.com/settings/organization/api-keys](https://platform.openai.com/settings/organization/api-keys)

---

好的！若你是使用 **Azure OpenAI** 而不是 OpenAI 官方 API，則需使用以下兩個環境變數來設定憑證資訊：

* `AZURE_OPENAI_API_KEY`：你的 Azure OpenAI 金鑰
* `AZURE_OPENAI_ENDPOINT`：你的 Azure OpenAI 端點網址（例如 `https://<your-resource-name>.openai.azure.com/`）

---

### 🔐 設定 Azure OpenAI 的 API 金鑰與端點

```python
os.environ["AZURE_OPENAI_API_KEY"]="" # 填上 api key
os.environ["AZURE_OPENAI_ENDPOINT"]=""
os.environ["OPENAI_API_VERSION"]="2024-12-01-preview" # 替換成你的 API 版本
os.environ["OPENAI_MODEL"]="text-embedding-3-large"
```


In [2]:
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.
import os

# Note. alternatively you can set a temporary env variable like this:
#os.environ["OPENAI_API_KEY"] = ""

os.environ["AZURE_OPENAI_API_KEY"]=""
os.environ["AZURE_OPENAI_ENDPOINT"]=""
os.environ["OPENAI_API_VERSION"]="2024-12-01-preview"
os.environ["OPENAI_MODEL"]="text-embedding-3-large"
#os.environ["OPENAI_MODEL"]="text-embedding-3-small"

if os.getenv("AZURE_OPENAI_API_KEY") is not None:
    print("AZURE_OPENAI_API_KEY is ready")
else:
    print("AZURE_OPENAI_API_KEY environment variable not found")

AZURE_OPENAI_API_KEY is ready


### 資料集

我們使用 Kaggle 上的資料集作為我們第一個 RAG 資料。

當你開啟這個 Kaggle 連結：
👉 [https://www.kaggle.com/datasets/xhlulu/covidqa/data?select=community.csv](https://www.kaggle.com/datasets/xhlulu/covidqa/data?select=community.csv)

你會看到這個資料集是來自 **[COVID-QA](https://github.com/deepset-ai/COVID-QA)** 專案的一部分，由使用者 `xhlulu` 上傳到 Kaggle，主要用來訓練與評估問答系統（Question Answering, QA）模型。

---

### 📁 資料集簡介：`covidqa`

這是一組基於 **COVID-19 文獻資料** 建立的問答資料集，目的是幫助開發者訓練自然語言理解（NLU）與問答系統，以回應與疫情相關的問題。

---

### 📄 檔案：`community.csv` 是什麼？

這個 `community.csv` 是該資料集中的一個檔案，根據名稱與用途，它大致包含從 COVID-19 社群討論（例如研究社群、論壇、問答平台等）中擷取出來的 **問答對（Question-Answer Pairs）**。

---

### 🔍 用途

* 用於訓練問答模型（如：BERT、GPT + 向量資料庫）。
* 適合建立 Retrieval-Augmented Generation (RAG) 系統。
* 可搭配向量嵌入技術（如 OpenAI Embedding + Qdrant）建立問答應用。

---

### 📌 延伸應用

你可以使用這份資料來：

* 建立 COVID-19 主題的 chatbot。
* 建構 QA pipeline，例如使用：

  * OpenAI Embedding 建立向量
  * 儲存至 Qdrant
  * 使用 LLM 查詢並生成回答

---

### 學生不需要下載

檔案已經存放到 github repository 中

In [3]:
import kagglehub

# Download a single file.
# https://www.kaggle.com/datasets/xhlulu/covidqa/data?select=community.csv
kagglehub.dataset_download('xhlulu/covidqa', path='community.csv')

'/home/jovyan/.cache/kagglehub/datasets/xhlulu/covidqa/versions/6/community.csv'

### 📥 載入資料

在這一節，我們將載入預先處理好的資料，**避免你用自己的 OpenAI API 金額重新計算文章的向量嵌入（embeddings）**。這讓你可以直接進行 Qdrant 儲存與查詢的實驗。

---

### ✅ 為什麼要用預處理資料？

* 計算文字嵌入會消耗 OpenAI 的使用額度（credits）。
* 若資料量大（例如整份 Wikipedia 的段落），嵌入計算時間與成本都不小。
* 使用事先嵌入好的資料，可以快速進行向量資料庫操作的示範與測試。

---

### 🧾 資料欄位範例：

資料集中有許多欄位，我們今天會使用到的欄位可能包括：

* `title`：問題文字，通常是自然語言表達的疑問句。
* `answer`：根據 `context` 找出的簡短回答。
* `question_id`：每筆資料的唯一識別碼。

In [4]:
import pandas as pd

article_df = pd.read_csv('/home/jovyan/.cache/kagglehub/datasets/xhlulu/covidqa/versions/6/community.csv')
article_df.head()

,question_id,title,question,answer_id,answer,answer_type,wrong_answer,wrong_answer_type,url,source
0,14057,Can pets catch the cold?,Last night I was drying my cat with a towel af...,14083,Yes they can. The viruses that cause a cold in...,Accepted,"That is a Priapulid worm, also known as a ""pen...",Random,biology.stackexchange.com,biomedical
1,89709,Is the Common Cold an Immune Overreaction?,It's my understanding that the majority of sym...,89712,Can someone die of the common cold?\n\nNo. \nT...,Accepted,"The dash (""-"") does not represent a negative c...",Random,biology.stackexchange.com,biomedical
2,89886,Air purifier agains bacteria and viruses?,We would buy a mobile air purifier in our home...,89887,The aforementioned filter will filter microbes...,Accepted,"It's a bleu ray gelyfish, don't tauch is becau...",Random,biology.stackexchange.com,biomedical
3,89929,Why are bats the source of dangerous coronavir...,Why do coronaviruses come from bats?\n\nI mean...,89944,\n The preponderance of links between bat and...,Accepted,"First of, depending on your definition of life...",Random,biology.stackexchange.com,biomedical
4,89938,How do bats survive their own coronaviruses?,How do bats survive their own coronaviruses (w...,89975,It's common for the reservoir host of a zoonot...,Accepted,"I think that ""career in synthetic biology"" and...",Random,biology.stackexchange.com,biomedical


In [5]:
article_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642 entries, 0 to 641
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   question_id        642 non-null    int64 
 1   title              642 non-null    object
 2   question           642 non-null    object
 3   answer_id          642 non-null    int64 
 4   answer             642 non-null    object
 5   answer_type        642 non-null    object
 6   wrong_answer       642 non-null    object
 7   wrong_answer_type  642 non-null    object
 8   url                642 non-null    object
 9   source             642 non-null    object
dtypes: int64(2), object(8)
memory usage: 50.3+ KB


In [6]:
article_df.describe(include="all")

,question_id,title,question,answer_id,answer,answer_type,wrong_answer,wrong_answer_type,url,source
count,642.000000,642,642,642.000000,642,642,642,642,642,642
unique,NaN,642,642,NaN,642,2,640,2,26,3
top,NaN,Can pets catch the cold?,Last night I was drying my cat with a towel af...,NaN,Yes they can. The viruses that cause a cold in...,Accepted,"Basically, the signal transduction pathway of ...",Random,travel.stackexchange.com,general
freq,NaN,1,1,NaN,1,335,2,544,119,300
mean,101172.771028,NaN,NaN,101244.221184,NaN,NaN,NaN,NaN,NaN,NaN
std,99158.636971,NaN,NaN,99156.996497,NaN,NaN,NaN,NaN,NaN,NaN
min,6235.000000,NaN,NaN,7195.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,35909.250000,NaN,NaN,35923.250000,NaN,NaN,NaN,NaN,NaN,NaN
50%,71596.500000,NaN,NaN,71608.500000,NaN,NaN,NaN,NaN,NaN,NaN
75%,147239.500000,NaN,NaN,147281.000000,NaN,NaN,NaN,NaN,NaN,NaN


---
# 以下的部分同學可以不用跑！
---

講師已經事前完成 embedding，包含 embedding 的檔案已經存在 `/home/jovyan/community_embedded_text_embedding_3_large.csv`

自己做 embedding 會消耗 embedding api rate limit，人數多的話可能沒有足夠的額度，讓其他同學被限流。

如果想要嘗試自己做 embedding，可以到 [4_RAG_DIY.ipynb](./4_RAG_DIY.ipynb) DIY 時一起進行

## 計算傳入的 string 的 token 長度

https://platform.openai.com/docs/guides/embeddings#how-can-i-tell-how-many-tokens-a-string-has-before-i-embed-it

In [7]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string("tiktoken is great!", "cl100k_base")

6

In [8]:
from openai import AzureOpenAI
openai_client = AzureOpenAI()

from tqdm import tqdm
from tenacity import retry, wait_random_exponential, stop_after_attempt

@retry(
    wait=wait_random_exponential(min=1, max=60),  # backoff 等待時間
    stop=stop_after_attempt(6),  # 最多重試 6 次
)
def embedding(input: str, model: str="text-embedding-3-large") -> str:
    response = openai_client.embeddings.create(
        input = input,
        model= model,
    )
    return response.data[0].embedding


顯示進度條地將每篇標題 title 轉換為向量嵌入（embedding）。

In [9]:
tqdm.pandas(desc="Generating title embeddings")
article_df['title_vector'] = article_df['title'].progress_apply(
    lambda x: embedding(x, model="text-embedding-3-large")
)

Generating title embeddings: 100%|██████████| 642/642 [03:20<00:00,  3.20it/s]


顯示進度條地將每篇標題 answer 轉換為向量嵌入（embedding）。

In [10]:
tqdm.pandas(desc="Generating answer embeddings")
article_df['answer_vector'] = article_df['answer'].progress_apply(
    lambda x: embedding(x, model="text-embedding-3-large")
)

Generating answer embeddings: 100%|██████████| 642/642 [04:12<00:00,  2.54it/s]


### 📥 輸出資料

講師將輸出預先處理好的資料，**避免你用自己的 OpenAI API 金額重新計算文章的向量嵌入（embeddings）**。這讓同學在後面的章節，可以使用 repository 內的 csv 檔案，直接進行 Qdrant 儲存與查詢的實驗。

In [11]:
article_df.to_csv("/home/jovyan/community_embedded_text_embedding_3_large.csv", index=False)

### ✅ 為什麼要用預處理資料？

* 計算文字嵌入會消耗 OpenAI 的使用額度（credits）。
* 若資料量大（例如整份 COVID-QA 的段落），嵌入計算時間與成本都不小。
* 使用事先嵌入好的資料，可以快速進行向量資料庫操作的示範與測試。

---

## 使用不同的 model 對相同資料集做 embedding

上面使用 OpenAI text-embedding-3-large 進行 embedding

下面使用 OpenAI 的其他 embedding model 進行 embedding

在這個 workshop 中同學可以嘗試感受不同 model 做 embedding 對於產出的變化，以及後面對於 RAG 系統的效能差異

---

## OpenAI 的 embedding models

以下是 OpenAI 目前提供的主要文字嵌入（embedding）模型的整理，包括維度、價格、效能評估指標（MIRACL 與 MTEB 分數）等資訊：

| 模型名稱                     | 維度（Dimensions） | 價格（每 1,000 tokens） | MIRACL 平均分數 | MTEB 平均分數 | 備註                                          |                                                                                              |
| ------------------------ | -------------- | ------------------ | ----------- | --------- | ------------------------------------------- | -------------------------------------------------------------------------------------------- |
| `text-embedding-3-large` | 3072           | \$0.00013          | 54.9%       | 64.6%     | 最新且最強大的嵌入模型，支援多語言，適合高準確度需求的應用。              |                                                                                              |
| `text-embedding-3-small` | 1536           | \$0.00002          | 44.0%       | 62.3%     | 相較於前代模型 `text-embedding-ada-002`，效能提升且價格更低。 |                                                                                              |
| `text-embedding-ada-002` | 1536           | \$0.00010          | 31.4%       | 61.0%     | 第二代嵌入模型，已被新模型取代，但仍可使用。                      |  |

**說明：**

* **維度（Dimensions）**：指嵌入向量的長度，較高的維度通常能捕捉更多語意資訊，但也會增加儲存與計算成本。
* **價格**：以每 1,000 個 tokens 為單位計費，`text-embedding-3-small` 的成本僅為 `text-embedding-ada-002` 的五分之一。
* **MIRACL 分數**：評估模型在多語言資訊檢索任務中的表現，分數越高表示效能越好。
* **MTEB 分數**：評估模型在英文任務（如分類、相似度評估等）中的表現，分數越高表示效能越好。([iThome][3], [iThome][2])

**其他資訊：**

* **維度調整**：`text-embedding-3` 系列模型支援透過 API 的 `dimensions` 參數調整輸出向量的維度。例如，`text-embedding-3-large` 可以縮減至 256 維，仍能維持高於 `text-embedding-ada-002` 的效能。
* **最大輸入長度**：所有模型的最大輸入長度為 8191 tokens。
* **知識截止日期**：這些模型的訓練資料截止於 2021 年 9 月，對於需要最新知識的應用可能有限制。([GitHub][1], [iThome][2])

根據你的應用需求，若追求高準確度且能接受較高成本，建議使用 `text-embedding-3-large`；若希望在成本與效能間取得平衡，`text-embedding-3-small` 是不錯的選擇；若已有既有系統使用 `text-embedding-ada-002`，可考慮升級以獲得更佳效能與成本效益。

---
## 底下使用 text-embedding-3-small

In [12]:
article_df_embedded = pd.read_csv('/home/jovyan/.cache/kagglehub/datasets/xhlulu/covidqa/versions/6/community.csv')
article_df_embedded.head()

,question_id,title,question,answer_id,answer,answer_type,wrong_answer,wrong_answer_type,url,source
0,14057,Can pets catch the cold?,Last night I was drying my cat with a towel af...,14083,Yes they can. The viruses that cause a cold in...,Accepted,"That is a Priapulid worm, also known as a ""pen...",Random,biology.stackexchange.com,biomedical
1,89709,Is the Common Cold an Immune Overreaction?,It's my understanding that the majority of sym...,89712,Can someone die of the common cold?\n\nNo. \nT...,Accepted,"The dash (""-"") does not represent a negative c...",Random,biology.stackexchange.com,biomedical
2,89886,Air purifier agains bacteria and viruses?,We would buy a mobile air purifier in our home...,89887,The aforementioned filter will filter microbes...,Accepted,"It's a bleu ray gelyfish, don't tauch is becau...",Random,biology.stackexchange.com,biomedical
3,89929,Why are bats the source of dangerous coronavir...,Why do coronaviruses come from bats?\n\nI mean...,89944,\n The preponderance of links between bat and...,Accepted,"First of, depending on your definition of life...",Random,biology.stackexchange.com,biomedical
4,89938,How do bats survive their own coronaviruses?,How do bats survive their own coronaviruses (w...,89975,It's common for the reservoir host of a zoonot...,Accepted,"I think that ""career in synthetic biology"" and...",Random,biology.stackexchange.com,biomedical


In [13]:
model="text-embedding-3-small"

article_small_df = pd.read_csv('/home/jovyan/.cache/kagglehub/datasets/xhlulu/covidqa/versions/6/community.csv')
tqdm.pandas(desc="Generating title embeddings")
article_small_df['title_vector'] = article_small_df['title'].progress_apply(
    lambda x: embedding(x, model=model)
)
tqdm.pandas(desc="Generating answer embeddings")
article_small_df['answer_vector'] = article_small_df['answer'].progress_apply(
    lambda x: embedding(x, model=model)
)

Generating answer embeddings: 100%|██████████| 642/642 [02:54<00:00,  3.68it/s]


In [14]:
article_small_df.to_csv("/home/jovyan/community_embedded_text_embedding_3_small.csv", index=False)

---
## 底下使用 text-embedding-ada-002

In [15]:
model="text-embedding-ada-002"

article_ada_df = pd.read_csv('/home/jovyan/.cache/kagglehub/datasets/xhlulu/covidqa/versions/6/community.csv')
tqdm.pandas(desc="Generating title embeddings")
article_ada_df['title_vector'] = article_ada_df['title'].progress_apply(
    lambda x: embedding(x, model=model)
)
tqdm.pandas(desc="Generating answer embeddings")
article_ada_df['answer_vector'] = article_ada_df['answer'].progress_apply(
    lambda x: embedding(x, model=model)
)

Generating answer embeddings: 100%|██████████| 642/642 [02:50<00:00,  3.77it/s]


In [16]:
article_ada_df.to_csv("/home/jovyan/community_embedded_text_embedding_ada_002.csv", index=False)

### 最後比較一下產出

1. vector 的長度
1. csv 的檔案大小 (bytes)

In [17]:
print(len(article_df.iloc[0]["title_vector"]))
print(len(article_small_df.iloc[0]["title_vector"]))
print(len(article_ada_df.iloc[0]["title_vector"]))

3072
1536
1536


In [18]:
print(os.path.getsize('/home/jovyan/community.csv'))

print(os.path.getsize('/home/jovyan/community_embedded_text_embedding_3_large.csv'))
print(os.path.getsize('/home/jovyan/community_embedded_text_embedding_3_small.csv'))
print(os.path.getsize('/home/jovyan/community_embedded_text_embedding_ada_002.csv'))

2609859
91062419
46527559
46801983


---
# 小結: Embedding

1. 使用 Kaggle 上整理好的 COVID-QA 資料集
2. 使用 openai embedding api 將資料集的 question 與 answer 的資料進行 embedding
3. 將 embedding 存成新的資料集，提供下一節 RAG 系統使用

### 檔案內容

- community.csv: kaggle 下載的原始資料集
- community_embedded_text_embedding_3_large.csv: 加入 text-embedding-3-large 嵌入
- community_embedded_text_embedding_3_small.csv: 加入 text-embedding-3-small 嵌入
- community_embedded_text_embedding_ada_002.csv: 加入 text-embedding-ada-002 嵌入

---

# 延伸問題

1. 這裡使用了 COVID-QA 資料集，整理得很好。現實中的專案通常沒有已經整理好的資料集，該怎麼辦？
2. 如何選擇 embedding model？例如幾個常用的 model 要如何選擇？
3. 如何選擇 database 或 vector database?

### 上面的問題，這個 workshop 都沒有答案，但都是現實中會遇到的問題

歡迎會後來找我聊天

*我還是沒有答案，但可以陪你聊天*
